In [1]:
import requests
from bs4 import BeautifulSoup as bs
from dataclasses import dataclass, asdict
import json


with open('title.html', 'r', encoding='utf-8') as f:
    html = f.read()
CATEGORY_IDS = {}
sp = bs(html, 'html.parser')
li = sp.find('ul', {'class': 'custom-horizontal-scrollbar'}).find_all('li')
for i in li[1:-1]:
    CATEGORY_IDS.update({i.get('id').split('-')[-1]: i.text.strip()})
CATEGORY_IDS

{'1': 'Trái Cây',
 '7': 'Rau Củ',
 '3': 'Hải Sản',
 '4': 'Thịt Heo',
 '2': 'Gà Vịt',
 '5': 'Bò Dê',
 '39': 'Món Canh',
 '38': 'Combo',
 '6': 'Trứng/Đậu',
 '22': 'Lẩu',
 '24': 'Món Chay',
 '51': 'Bếp',
 '77': 'Sữa/Pho Mai',
 '10': 'Đồ Uống',
 '12': 'Gia Vị',
 '13': 'Lương Thực',
 '11': 'Bánh'}

In [49]:
@dataclass
class Product_1:
    id: str
    name: str
    sub_category: str
    category_name: str
    url: str
    image_url: str
    
def write_to_json_file(data, file_name):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
def read_from_json_file(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def get_kindId_infos(kind_id, has_sub = False, sub_categories = []):
    if has_sub and len(sub_categories) == 0:
        return False
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'vi-VN,vi;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json',
        'Origin': 'https://www.cooky.vn',
        'Referer': 'https://www.cooky.vn/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
        'X-Cooky-City': '217',
        'X-Cooky-Client-Type': 'web',
        'X-Cooky-Client-Version': '5.4.2',
        'X-Cooky-Seller': '4',
        'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    params = {
        'checksum': 'd4f1130ff57fd9ccb0d732d425137a16'if has_sub else '7018ca9a124ba81a70d5bd6770c9148d'
    }

    json_data = {
        'sortType': 5,
        'isJustOrder': False,
        'categories': kind_id,
        'subCategories': sub_categories,
    }

    data = requests.post('https://marketapi.cooky.vn/discovery/v6/search/product', params=params, headers=headers, json=json_data).json()['data']
    return data

def get_productId_list(response_from_get_kindId_infos):
    return [i['productIds'] for i in response_from_get_kindId_infos]

def get_product_infos(ids: list):
    id_string = f'[{", ".join(list(map(str, ids)))}]'
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'vi-VN,vi;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5',
        'Connection': 'keep-alive',
        'Content-Type': 'text/plain;charset=UTF-8',
        'Origin': 'https://www.cooky.vn',
        'Referer': 'https://www.cooky.vn/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
        'X-Cooky-App-Id': '1001',
        'X-Cooky-City-Id': '217',
        'X-Cooky-Client-Id': '0101',
        'X-Cooky-Client-Language': 'vi',
        'X-Cooky-Client-Type': '1',
        'X-Cooky-Client-Version': '5.2',
        'X-Cooky-Seller': '4',
        'X-Cooky-Timestamp': '1687834687596',
        'sec-ch-ua': '".Not/A)Brand";v="99", "Google Chrome";v="103", "Chromium";v="103"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    payload = '{"ids":' + id_string +'}'

    response = requests.post('https://app-api.cooky.vn/api/product/get_basic_infos', headers=headers, data=payload)
    return response.json()['reply']['product_infos']

def parse_info_from_product_list(id_list, category_name: str, sub_category_name: str):
    infos = []
    for i in id_list:
        p = Product_1(id = i['id'], 
                      name = i['name'],
                      sub_category=sub_category_name, 
                      category_name = category_name,
                      url = i['url'],
                      image_url = i['photos'][0][0]['url'])
        infos.append(asdict(p))
    return infos  

def get_sub_catogories():
    url = 'https://marketapi.cooky.vn/shared/v1.5/category/list/product'
    data = requests.get(url).json()['data']
    data.pop(0)
    res = {i['name']: i['subCategories'] for i in data} 
    return res
sub_categories = get_sub_catogories()
sub_categories

{'Trái Cây': [{'id': 263, 'name': 'Dưa'},
  {'id': 264, 'name': 'Thanh Long'},
  {'id': 284, 'name': 'Kiwi'},
  {'id': 266, 'name': 'Thơm'},
  {'id': 267, 'name': 'Nho'},
  {'id': 261, 'name': 'Xoài'},
  {'id': 481, 'name': 'Sapoche'},
  {'id': 278, 'name': 'Dừa'},
  {'id': 228, 'name': 'Sầu Riêng'},
  {'id': 317, 'name': 'Trái Cây Vitamin C'},
  {'id': 274, 'name': 'Mít'},
  {'id': 277, 'name': 'Bơ'},
  {'id': 271, 'name': 'Mận'},
  {'id': 265, 'name': 'Ổi'},
  {'id': 262, 'name': 'Táo'},
  {'id': 268, 'name': 'Cherry'},
  {'id': 291, 'name': 'Củ Sắn'},
  {'id': 289, 'name': 'Chôm Chôm'},
  {'id': 273, 'name': 'Cóc'},
  {'id': 269, 'name': 'Vải'},
  {'id': 276, 'name': 'Quýt'},
  {'id': 279, 'name': 'Chuối'},
  {'id': 272, 'name': 'Đào'},
  {'id': 280, 'name': 'Cam'},
  {'id': 545, 'name': 'Muối'}],
 'Rau Củ': [{'id': 518, 'name': 'Rau Củ Quả Tươi'},
  {'id': 422, 'name': 'Rau Gia Vị'},
  {'id': 74, 'name': 'Củ & Hạt'},
  {'id': 215, 'name': 'Măng'},
  {'id': 75, 'name': 'Quả Trái'},


In [53]:
totalProduct_1 = []
for i in CATEGORY_IDS:
    category = sub_categories[CATEGORY_IDS[i]]
    for c in category:
        kind_id_infos = get_kindId_infos([i], True, [c['id']])
        ids = get_productId_list(kind_id_infos)
        for small_id in ids[:]:
            l = len(small_id)
            if l <= 100:
                a = get_product_infos(small_id)
                totalProduct_1 += parse_info_from_product_list(a, CATEGORY_IDS[i], c['name'])
            else:
                length = []
                while l > 100:
                    a = get_product_infos(small_id[:100])
                    totalProduct_1 += parse_info_from_product_list(a, CATEGORY_IDS[i])
                    small_id = small_id[100:]
                    l = len(small_id)
write_to_json_file(totalProduct_1, 'totalProduct_1.json')